## 1. Settings of Notebook

In [ ]:
import time

URL = "https://deploy-preview-154--wave-js.netlify.app/" # or "http://localhost:3002/"
JSON_FOLDER = "uploads"

## 2. Loading of materials

In [ ]:
import os
import json

def load_json_files_from_folder(folder_path):
    json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
    json_data = []
    file_names = []

    for json_file in json_files:
        file_path = os.path.join(folder_path, json_file)
        with open(file_path, 'r') as f:
            data = json.load(f)
            json_data.append(data)
            file_names.append(os.path.splitext(json_file)[0])

    return json_data, file_names

json_data, file_names = load_json_files_from_folder(JSON_FOLDER)

materials_settings = []
for data in json_data:
    materials_settings.append({
        "material_json": data,
        "name": file_names[json_data.index(data)] # name of the file
        # "name": data["name"] # name of the material
    })

#
# material_json_1 = {
#     "name": "New Material",
#     "basis": {
#         "elements": [
#             {"id": 0, "value": "Si"},
#             {"id": 1, "value": "Si"}
#         ],
#         "coordinates": [
#             {"id": 0, "value": [0, 0, 0]},
#             {"id": 1, "value": [0.25, 0.25, 0.25]}
#         ],
#         "units": "crystal",
#         "cell": [
#             [3.34892, 0, 1.9335],
#             [1.116307, 3.157392, 1.9335],
#             [0, 0, 3.867]
#         ],
#         "constraints": []
#     },
#     "lattice": {
#         "a": 3.867, "b": 3.867, "c": 3.867,
#         "alpha": 60, "beta": 90, "gamma": 90,
#         "units": {"length": "angstrom", "angle": "degree"},
#         "type": "FCC",
#         "vectors": {
#             "a": [3.34892, 0, 1.9335],
#             "b": [1.116307, 3.157392, 1.9335],
#             "c": [0, 0, 3.867],
#             "alat": 1,
#             "units": "angstrom"
#         }
#     },
#     "isNonPeriodic": False
# }
#
# material_json_2 = {
#     "name": "Another Material",
#     "basis": {
#         "elements": [
#             {"id": 0, "value": "Si"},
#             {"id": 1, "value": "O"},
#             {"id": 2, "value": "Si"},
#             {"id": 3, "value": "O"},
#             {"id": 4, "value": "Si"},
#             {"id": 5, "value": "O"}
#         ],
#         "coordinates": [
#             {"id": 0, "value": [0, 0, 0]},
#             {"id": 1, "value": [0.15, 0.15, 0.15]},
#             {"id": 2, "value": [0.15, 0.25, 0.25]},
#             {"id": 3, "value": [0.15, 0.25, 0.10]},
#             {"id": 4, "value": [0.5, 0.5, 0.5]},
#             {"id": 5, "value": [0.55, 0.55, 0.60]}
#         ],
#         "units": "crystal",
#         "cell": [
#             [7.0, 0, 0],
#             [0, 7.0, 0],
#             [0, 0, 7.0]
#         ],
#         "constraints": []
#     },
#     "lattice": {
#         "a": 7.0, "b": 7.0, "c": 7.0,
#         "alpha": 90, "beta": 90, "gamma": 90,
#         "units": {"length": "angstrom", "angle": "degree"},
#         "type": "CUB",
#         "vectors": {
#             "a": [7.0, 0, 0],
#             "b": [0, 7.0, 0],
#             "c": [0, 0, 7.0],
#             "alat": 1,
#             "units": "angstrom"
#         }
#     },
#     "isNonPeriodic": False
# }
#
# materials_settings = [{
#     "material_json": material_json_1,
#     "name": "New Material 1 (Si2).gif"
# },
# {"material_json": material_json_2,
#     "name": "New Material 2 (SiO).gif"
# }]

## 3. Definitions

In [ ]:
from IPython.display import display, IFrame, Javascript, HTML
import json

import time
import os


def send_message_to_iframe(message):
    """
    Creates and displays an iframe pointing to `url`,
    then sends `message` to the iframe via postMessage.
    """
    js_code = f"""
    (function() {{
        const iframe = document.querySelector('iframe');
        if (iframe && iframe.contentWindow) {{
            try {{
                iframe.contentWindow.postMessage({json.dumps(message)}, '*');
                console.log('Message sent to iframe');
            }} catch (error) {{
                alert('Error sending message to iframe. See console for more information.', error);
            }}
        }} else {{
            console.error('Iframe not found or not ready');
        }}
    }})();
    """
    display(Javascript(js_code))


def set_material_in_iframe(material_json):
    """
    Uses send_message_to_iframe to send a material configuration
    under the "material" key to the iframe at `url`.
    """
    send_message_to_iframe({"material": material_json})


def record_gif(filename, rotation_speed=60, frame_duration=0.05):
    """
    Uses send_message_to_iframe to send a message to the iframe at `url`
    to start recording a GIF of the visualization with the specified parameters.
    """
    message = {
        "action": "handleStartGifRecording",
        "parameters": [filename, rotation_speed, frame_duration]
    }
    send_message_to_iframe(message)

def set_camera( pos=(15, 15, 15), target=(0, 0, 0)):
    func_str = f"camera.position.set({pos[0]},{pos[1]},{pos[2]})"
    message = {
        "action": "doWaveFunc",
        "parameters": [func_str]
    }
    send_message_to_iframe(message)
    # func_str = f"camera.lookAt({target[0]},{target[1]},{target[2]})"
    # message = {
    #     "action": "doFunc",
    #     "parameters": [func_str]
    # }
    # send_message_to_iframe(message)

def set_camera_to_fit_cell():
    message = {
        "action": "handleSetCameraToFitCell",
        "parameters": [None]
    }
    send_message_to_iframe(message)

def toggle_bonds():
    message = {
        "action": "handleToggleBonds",
        "parameters": [None]
    }
    send_message_to_iframe(message)

def handle_reset_viewer():
    message = {
        "action": "handleResetViewer",
        "parameters": [None]
    }
    send_message_to_iframe(message)

def test_do_func():
    func_str = "camera.position.set(0,16,0)"
    message = {
        "action": "doWaveFunc",
        "parameters": [func_str]
    }
    send_message_to_iframe(message)


## 4. Gif Generation

In [ ]:
# Example usage with a material configuration:
iframe = IFrame(src=URL, width="600", height="600")
display(iframe)


In [ ]:

for material_settings in materials_settings:
    GIF_NAME =  material_settings["name"]+".gif" or material_settings["material_json"]["name"] + ".gif"
    handle_reset_viewer()
    time.sleep(1)
    set_material_in_iframe(material_settings["material_json"])
    time.sleep(1)
    toggle_bonds()
    time.sleep(1)
    # set_camera() # doesn't work
    time.sleep(1)
    set_camera_to_fit_cell()
    time.sleep(1)
    record_gif( GIF_NAME, 60, 0.05)
    # We should wait until the gif is generated and saved before moving to the next material
    parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
    gif_path = os.path.join(parent_dir, GIF_NAME)
    print(f"Waiting for gif to be generated in {gif_path}")

    while not os.path.exists(gif_path):
        time.sleep(1)
    else:
        print(f"Generated gif for {GIF_NAME}")
    # time.sleep(30)
